# 0 Imports

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

import seaborn as sns
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

import scipy.io
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
import torch.nn as nn
from torch.nn import BatchNorm2d
from torch.nn import Dropout2d
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Adam
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
 
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torchinfo import summary

from livelossplot import PlotLosses

np.random.seed(0) 
torch.manual_seed(0)
import random
random.seed(0)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Constants

# path para guardar o dataset
PATH = './'
PATH_TRAIN_CSV = './data/train.csv'
PATH_TRAIN_IMG = './data/train_data.mat'
PATH_TEST_CSV = './data/test.csv'
PATH_TEST_IMG = './data/test_data.mat'

BATCH_SIZE = 28

device = torch.device("cuda")

# 1 Exploração e Preparação dos Dados

### Visualização dos dados

In [ ]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    # print(train_mat.keys())
    # print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    # print(train_np.shape)
    # print(test_np.shape)
    return  train_np, test_np


def load_data(path_train_csv, path_train_img, path_test_csv, path_test_img):
    train_csv = pd.read_csv(path_train_csv, header=0)
    test_csv = pd.read_csv(path_test_csv, header=0)
    train_img, test_img = get_data_from_mat(path_train_img, path_test_img)
    # train = train_csv + train_img
    # test = test_csv + test_img
    # return train, test
    return train_csv, test_csv, train_img, test_img


def visualize(image):
    #plt.figure("sample", (12, 6))
    #plt.subplot(1, 2, 1)
    plt.imshow(image, cmap="gray")    
    #plt.subplot(1, 2, 2)
    #plt.imshow(image, cmap="gray")
    plt.show()      

def show_ds(ds):
    print("ds shape:",ds.shape)
    print("ds max:",np.max(ds))
    print("ds min:",np.min(ds))
    print("ds average:",np.average(ds))
    @interact
    def visualize_set(scan_index=(0,len(ds)-1)):
        #print(scan_index)
        visualize(ds[scan_index,:,:])

In [ ]:
train_csv, test_csv, train_img, test_img = load_data(PATH_TRAIN_CSV, PATH_TRAIN_IMG, PATH_TEST_CSV, PATH_TEST_IMG)

print("Data de treino:")
print(train_csv)
show_ds(train_img)
print("Data de Teste:")
print(test_csv)
show_ds(test_img)

Data de treino:
      id  age  sex  education
0      1   13    1          7
1      2   14    0          8
2      3   15    1          9
3      4   15    1          9
4      5   15    1          9
..   ...  ...  ...        ...
107  108   77    1          4
108  109   67    0          4
109  110   55    0          4
110  111   76    1          3
111  112   69    0          4

[112 rows x 4 columns]
ds shape: (112, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024116160549305543


interactive(children=(IntSlider(value=55, description='scan_index', max=111), Output()), _dom_classes=('widget…

Data de Teste:
    id  sex  education
0    1    0         13
1    2    0         11
2    3    1          9
3    4    1         13
4    5    0         12
5    6    0         17
6    7    0          9
7    8    0          4
8    9    1          9
9   10    1          4
10  11    1         14
11  12    0          9
12  13    1          2
13  14    1          5
14  15    1          9
15  16    1         11
16  17    1          9
17  18    0         17
18  19    1         15
19  20    1         14
20  21    1          4
21  22    0          4
22  23    1          4
23  24    0          0
24  25    1          2
25  26    1          4
26  27    1          3
27  28    1          4
ds shape: (28, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024994026906171023


interactive(children=(IntSlider(value=13, description='scan_index', max=27), Output()), _dom_classes=('widget-…

### Preparação dos dados

In [ ]:
def fix_sex(csv):
    female = csv['sex']
    male = []
    for person in female:
        if person==1:
            male.append(0)
        else:
            male.append(1)
    csv.drop('sex', axis='columns', inplace=True)
    csv['Female']=female
    csv['Male']=male
    return csv
    
def img_to_list(img):
    tamanho = len(img)
    lista = []
    # triangular inferior sem diagonal
    for linha in range(tamanho):
        for coluna in range(linha):
            lista.append(img[linha][coluna])
    return lista

def imgs_to_matrix(imgs):
    matrix = []
    for img in imgs:
        matrix.append(img_to_list(img))
    return matrix

def remove_null_columns(matrix1, matrix2):
    columns_to_remove = []
    m1_row_len = len(matrix1)
    m2_row_len = len(matrix2)
    column_len = len(matrix1[0])
    for column in range(column_len):
        all_zero = True
        for row in range(m1_row_len):
            if matrix1[row][column]!=0:
                all_zero=False
        for row in range(m2_row_len):
            if matrix2[row][column]!=0:
                all_zero=False
        if all_zero:
            columns_to_remove.append(column)

    brain_activity_index = list(range(column_len))
    for column in columns_to_remove[::-1]:
        brain_activity_index.pop(column)
        for line in range(m1_row_len):
            matrix1[line].pop(column)
        for line in range(m2_row_len):
            matrix2[line].pop(column)
    return matrix1, matrix2, brain_activity_index

def join_data(train_csv, test_csv, train_img, test_img):
    train_matrix = imgs_to_matrix(train_img)
    test_matrix = imgs_to_matrix(test_img)
    train_clean_matrix, test_clean_matrix, brain_activity_index = remove_null_columns(train_matrix, test_matrix)
    train_data  = pd.concat([train_csv, pd.DataFrame(train_clean_matrix)], axis=1)
    train_data .columns = list(train_csv.columns) + [f'rel-{brain_activity_index[i]}' for i in range(len(brain_activity_index))]
    test_data = pd.concat([test_csv, pd.DataFrame(test_clean_matrix)], axis=1)
    test_data.columns = list(test_csv.columns) + [f'rel-{brain_activity_index[i]}' for i in range(len(brain_activity_index))]
    return train_data , test_data, brain_activity_index

In [ ]:
train_csv, test_csv, train_img, test_img = load_data(PATH_TRAIN_CSV, PATH_TRAIN_IMG, PATH_TEST_CSV, PATH_TEST_IMG)
rain_csv = fix_sex(train_csv)
test_csv = fix_sex(test_csv)
train_data, test_data, brain_activity_index = join_data(train_csv, test_csv, train_img, test_img)
print(train_data)
print(test_data)
train_data= train_data.drop('id', axis=1)
test_data= test_data.drop('id', axis=1)
train_data.to_csv('./data/train_joined.csv', index=False)
test_data.to_csv('./data/test_joined.csv', index=False)

      id  age  education  Female  Male     rel-1  rel-3     rel-4     rel-5  \
0      1   13          7       1     0  0.131525    0.0  0.136950  0.000000   
1      2   14          8       0     1  0.117704    0.0  0.132998  0.037714   
2      3   15          9       1     0  0.155806    0.0  0.165429  0.022278   
3      4   15          9       1     0  0.145214    0.0  0.162493  0.000000   
4      5   15          9       1     0  0.161360    0.0  0.172043  0.017686   
..   ...  ...        ...     ...   ...       ...    ...       ...       ...   
107  108   77          4       1     0  0.215767    0.0  0.184942  0.019446   
108  109   67          4       0     1  0.143005    0.0  0.124350  0.000000   
109  110   55          4       0     1  0.116085    0.0  0.156406  0.020947   
110  111   76          3       1     0  0.156654    0.0  0.166613  0.046464   
111  112   69          4       0     1  0.114586    0.0  0.128877  0.045583   

        rel-8  ...  rel-3975  rel-3977  rel-3979  r

In [ ]:
# criar as funções de transformação
train_transform = Compose(
        [ToTensor(), 
         Normalize(mean=(0.1307,), std=(0.3081,)) # parâmetros por canal da imagem, image = (image - mean) / std
        ])
test_transform = Compose(
        [ToTensor(), 
         Normalize(mean=(0.1307,), std=(0.3081,)) # parâmetros por canal da imagem, image = (image - mean) / std
        ])

In [ ]:
# buscar o dataset utilizando os CSVs e uma classe para o dataset

# definição classe para o dataset
class CSVDataset(Dataset):
  # ler o dataset
  def __init__(self, path, transform=None):
    self.transform = transform
    # ler o ficheiro csv para um dataframe
    df_set = pd.read_csv(path, header=0)
    # separar os inputs e os outputs
    self.x = df_set.values[:, 1:]
    self.y = df_set.values[:, 0]

    # garantir que os inputs e labels sejam floats
    # self.x = self.x.astype('float32')
    # self.y = self.y.astype('long')
    #print(self.x.shape)
    #print(self.y.shape)
        
  # número de casos no dataset
  def __len__(self):
    return len(self.x)

  # retornar um caso
  def __getitem__(self, idx):
    label = self.y[idx]
    image = self.x[idx]       
    if self.transform is not None:
      image = self.transform(image)
    return image, label
    
  # retornar índices para casos de treino e de teste em formato imagem
  def get_TensorDataset(self):
    x = self.x.reshape(len(self.x), 1, 1205)
    xmax, xmin = x.max(), x.min()
    x = (x - xmin)/(xmax - xmin)
    x  = torch.from_numpy(np.array(x)).float()
    y  = torch.from_numpy(np.array(self.y)).type(torch.LongTensor)
    cases = torch.utils.data.TensorDataset(x,y)
    return cases 
  
  def get_TensorDataset_test(self):
    x = self.x.reshape(len(self.x), 1, 1204)
    xmax, xmin = x.max(), x.min()
    x = (x - xmin)/(xmax - xmin)
    x  = torch.from_numpy(np.array(x)).float()
    y  = torch.from_numpy(np.array(self.y)).type(torch.LongTensor)
    cases = torch.utils.data.TensorDataset(x,y)
    return cases 

#com holdout
def prepare_data_loaders(path_train, path_test):
  train_data = CSVDataset(path_train,transform=train_transform)
  test_data = CSVDataset(path_test,transform=test_transform)
  train_data = train_data.get_TensorDataset()
  test_data = test_data.get_TensorDataset_test()
  train_size = int(0.8 * len(train_data))
  val_size = len(train_data) - train_size
  train_data, validation = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))
    
  train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
  val_dl = DataLoader(validation, batch_size=BATCH_SIZE, shuffle=True)
  test_dl = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
  train_dl_all = DataLoader(train_data, batch_size=len(train_data), shuffle=False)
  val_dl_all = DataLoader(validation, batch_size=len(validation), shuffle=True)
  test_dl_all = DataLoader(test_data, batch_size=len(test_data), shuffle=False)
  return train_dl, val_dl, test_dl, train_dl_all, val_dl_all, test_dl_all

# preparar os dados
path_train = './data/train_joined.csv'
path_test = './data/test_joined.csv'
train_dl, val_dl, test_dl, train_dl_all, val_dl_all, test_dl_all = prepare_data_loaders(path_train, path_test)

# 2 Definir o Modelo

In [ ]:
# CNN Model2: Sequential (Conv, ReLU, MaxPool), Sequential, Linear
class CNNModel_2(Module):
    def __init__(self):
        super(CNNModel_2, self).__init__()
        self.layer1 = Sequential(Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0),
                                 ReLU(),
                                 MaxPool2d(kernel_size=2)
                                )
        self.layer2 = Sequential(Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
                                 ReLU(),
                                 MaxPool2d(kernel_size=2)
                                )       
        self.fc1 = Linear(in_features=32 * 5 * 5, out_features=10)  # Fully connected 1
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) #Flatten
        out = self.fc1(out) #Dense
        return out
    
model = CNNModel_2()
print(summary(model, input_size=(BATCH_SIZE, 1,28,28), verbose=0)) #verbose=2 Show weight and bias layers in full detail
    
# CNN Model4: Sequential (Conv, BatchNorm, ReLU, MaxPool, Dropout), Linear, Linear
# class CNNModel_4(Module):
#     def __init__(self):
#         super(CNNModel_4, self).__init__()
#         self.layer1 = Sequential(nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=0),
#                                     BatchNorm2d(32),
#                                     ReLU(),
#                                     MaxPool2d(2),
#                                     Dropout2d(0.2)
#                                     )
#         self.fc1 = Linear(in_features=BATCH_SIZE*12*12, out_features=128) # 16928
#         self.fc2 = Linear(in_features=128, out_features=10)
        
#     def forward(self, x):
#         out = self.layer1(x)
#         out = out.view(out.size(0), -1)
#         out = self.fc1(out)
#         out = self.fc2(out)
#         return out
    
# model = CNNModel_4()
# print(summary(model, input_size=(BATCH_SIZE, 1,28,28), verbose=0)) #verbose=2 Show weight and bias layers in full detail

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel_2                               [28, 10]                  --
├─Sequential: 1-1                        [28, 16, 13, 13]          --
│    └─Conv2d: 2-1                       [28, 16, 26, 26]          160
│    └─ReLU: 2-2                         [28, 16, 26, 26]          --
│    └─MaxPool2d: 2-3                    [28, 16, 13, 13]          --
├─Sequential: 1-2                        [28, 32, 5, 5]            --
│    └─Conv2d: 2-4                       [28, 32, 11, 11]          4,640
│    └─ReLU: 2-5                         [28, 32, 11, 11]          --
│    └─MaxPool2d: 2-6                    [28, 32, 5, 5]            --
├─Linear: 1-3                            [28, 10]                  8,010
Total params: 12,810
Trainable params: 12,810
Non-trainable params: 0
Total mult-adds (M): 18.97
Input size (MB): 0.09
Forward/backward pass size (MB): 3.29
Params size (MB): 0.05
Estimated Total Size (MB): 3.43


In [ ]:
class SingleRowCNN(Module):
    def __init__(self, input_size, num_classes):
        super(SingleRowCNN, self).__init__()

        self.conv = nn.Conv1d(in_channels=input_size, out_channels=16, kernel_size=3)


        self.relu = nn.ReLU()


        self.fc = nn.Linear(19248, num_classes)

    def forward(self, x):
        # Apply convolutional layer
        x = self.conv(x)


        x = self.relu(x)


        x = x.view(x.size(0), -1)


        x = self.fc(x)

        return x


input_size = 1  # Number of features in the input data
num_classes = 10  # Number of output classes


model = SingleRowCNN(input_size, num_classes)

# 3 Treinar o Modelo

In [ ]:
def train_model(h5_file, train_dl, val_dl, model, criterion, optimizer):
    liveloss = PlotLosses()
    for epoch in range(EPOCHS):
        logs = {} # para o livelossplot
        # Train phase
        model.train() 
        running_loss  = 0.0
        running_corrects  = 0.0
        for batch_i, (inputs, labels) in enumerate(train_dl): 
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach() * inputs.size(0)
            #acc = accuracy_score(labels.numpy(), np.argmax(outputs.detach().numpy(), axis=1))
            _, preds = torch.max(outputs, 1) # Get predictions from the maximum value
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(train_dl.dataset)
        epoch_acc = running_corrects.float() / len(train_dl.dataset)
        logs['loss'] = epoch_loss.item()
        logs['accuracy'] = epoch_acc.item()
        
        # Validation phase
        model.eval()
        running_loss  = 0.0
        running_corrects  = 0.0
        for inputs, labels in val_dl: 
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.detach() * inputs.size(0)
            _, preds = torch.max(outputs, 1) # Get predictions from the maximum value
            #acc = accuracy_score(labels.numpy(), np.argmax(outputs.detach().numpy(), axis=1))
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_dl.dataset)
        epoch_acc = running_corrects.float() / len(val_dl.dataset)
        logs['val_loss'] = epoch_loss.item()
        logs['val_accuracy'] = epoch_acc.item()
        #print(f'Epoch {epoch:03}: | Loss: {epoch_loss/len(train_dl):.5f} | Acc: {epoch_acc/len(train_dl):.3f}')      
        liveloss.update(logs)
        liveloss.send()
    torch.save(model,h5_file) # para gravar o modelo no final do treino

In [ ]:
######### CNNModel_2 ################
# model = CNNModel_2()
model.cuda()
device = torch.device("cuda")

In [ ]:
# treino do modelo CNNModel_2
# for batch in train_dl:
#     print(batch)
EPOCHS = 15
LEARNING_RATE = 0.001
criterion = CrossEntropyLoss() # neste caso implementa a sparse_categorical_crossentropy #nn.CrossEntropyLoss accepts ground truth labels directly as integers #in [0, N_CLASSES[ (no need to onehot encode the labels)
optimizer = SGD(model.parameters(), lr=LEARNING_RATE) 
#optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
starttime = time.perf_counter()
train_model('CNNModel_2.pth', train_dl, val_dl, model, criterion, optimizer)
endtime = time.perf_counter()
print(f"Tempo gasto: {endtime - starttime} segundos")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# 4 Avaliar o Modelo

# 5 Testar o Modelo